In [1]:
import shadow_reconstruction
import numpy
from matplotlib import pyplot
import os

In [2]:
def make_perms(n):
    return itertools.permutations(numpy.arange(n))

# Simple to start

$$ \vert \psi \rangle = \frac{|01\rangle + |10\rangle}{\sqrt{2}}$$

In [3]:
zz = numpy.array([[0.,1.,1.,0.]])
zz /= numpy.linalg.norm(zz)

rho = zz.T @ zz
# pyplot.imshow(rho)
# pyplot.show()

print(numpy.trace(rho))
rho = shadow_reconstruction.DensityMatrix.new_mixed_sparse(rho.astype(numpy.complex128))

0.9999999999999998


# Bigger

$$|\psi\rangle = \frac{|1000> + |0001>}{\sqrt{2}}$$

In [4]:
# (01 + 10)00
zz = numpy.zeros((1, 1<<4), dtype=numpy.complex128)
zz[0, 0b1000] = 1.0
zz[0, 0b0100] = 1.0
zz /= numpy.linalg.norm(zz)
rho = zz.T @ zz
# print(numpy.trace(rho))
rho /= numpy.trace(rho)
rho = shadow_reconstruction.DensityMatrix.new_mixed_sparse(rho.astype(numpy.complex128))
rho.expectation_string('ZIII'), rho.expectation_string('ZZII')

(0j, (-1+0j))

In [5]:
rho = shadow_reconstruction.DensityMatrix.new_pure_dense(zz[0,:].astype(numpy.complex128))
rho.expectation_string('ZIII'), rho.expectation_string('ZZII')

(0j, (-0.9999999999999998+0j))

In [6]:
exp = shadow_reconstruction.Experiment(4)

In [7]:
# if os.path.exists('larger_samples_pure.dat'):
#     print("Loading...")
#     samples = shadow_reconstruction.Samples.load_from('larger_samples_pure.dat')
# else:
print("Sampling...")
samples = exp.sample(rho, 1_000_000)
print("Saving...")

Sampling...
Saving...


In [16]:
recon = shadow_reconstruction.Reconstruction(False)

In [17]:
recon.use_dumb_estimator()
recon.estimate_string("ZIII", samples), rho.expectation_string('ZIII')

((0.00048560000000010315+0j), 0j)

In [18]:
recon.use_smart_estimator()
recon.estimate_string("ZIII", samples), rho.expectation_string('ZIII')

((0.00048560000000011085+0j), 0j)

In [19]:
recon.use_inversion_by_parts_estimator()
recon.estimate_string("ZIII", samples), rho.expectation_string('ZIII')

((0.0014220000000000025+0j), 0j)

In [11]:
recon.use_smart_estimator()
recon.estimate_string("ZZII", samples), rho.expectation_string('ZZII')

((-1.0006146000000018+0j), (-0.9999999999999998+0j))

In [20]:
recon.use_inversion_by_parts_estimator()
recon.estimate_string("ZZII", samples), rho.expectation_string('ZZII')

((-0.999528+0j), (-0.9999999999999998+0j))

In [12]:
recon.use_smart_estimator()
recon.estimate_string("ZIZI", samples), rho.expectation_string('ZIZI')

((0.00018539999999999987+0j), 0j)

In [21]:
recon.use_inversion_by_parts_estimator()
recon.estimate_string("ZIZI", samples), rho.expectation_string('ZIZI')

((0.000321+0j), 0j)

In [13]:
recon.use_smart_estimator()
recon.estimate_string("+-II", samples), rho.expectation_string('+-II')

((0.4990094999999999-0.00018449999999999977j), (0.4999999999999999+0j))

In [22]:
recon.use_inversion_by_parts_estimator()
recon.estimate_string("+-II", samples), rho.expectation_string('+-II')

((0.4988745000000001-0.0012600000000000003j), (0.4999999999999999+0j))

In [14]:
recon.use_smart_estimator()
recon.estimate_string("-++-", samples), rho.expectation_string('-++-')

(0j, 0j)

In [23]:
recon.use_inversion_by_parts_estimator()
recon.estimate_string("-++-", samples), rho.expectation_string('-++-')

(0j, 0j)

# Check filtering

In [15]:
recon = shadow_reconstruction.Reconstruction(False)

meas = numpy.round(recon.estimate_string_for_each_sample("+-II", samples).real, decimals=5)
vals, counts = numpy.unique(meas, return_counts=True)
vals, counts, sum(counts), numpy.var(meas)

(array([0. , 4.5]), array([889109, 110891]), 1000000, 1.9965322689097507)

In [16]:
recon.estimate_string("+-II", samples)

(0.4990094999999999-0.00018450000000000023j)

In [17]:
recon = shadow_reconstruction.Reconstruction(True)

meas = numpy.round(recon.estimate_string_for_each_sample("+-II", samples).real, decimals=5)
vals, counts = numpy.unique(meas, return_counts=True)
vals, counts, sum(counts), numpy.var(meas)

(array([0. , 1.5]), array([222091, 110891]), 332982, 0.4997677901919403)

In [18]:
recon.estimate_string("+-II", samples)

(0.49953601095554695-0.00018469466818026215j)